In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = './subsample/'
    TEST_DIR = './test/'

    # use this for full dataset
    train_images = [TRAIN_ORIGINAL_DIR + i for i in os.listdir(TRAIN_ORIGINAL_DIR)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [3]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

nb_classes = 4

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/8000
[INFO] processed 2000/8000
[INFO] processed 3000/8000
[INFO] processed 4000/8000
[INFO] processed 5000/8000
[INFO] processed 6000/8000
[INFO] processed 7000/8000
[INFO] processed 8000/8000
[INFO] processed 240/240
(8000, 128, 128, 3)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [

In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]    
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
#                 batch_x = np.array(seq.augment_images(np.array(batch_x,dtype = uint8)),dtype = float32)
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 50
batch_size = 80
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 12.58, total 50 epochs ptime: 647.16
#  [*] Training finished!
#  [*] Best Epoch:  17 , Accuracy:  0.9541666507720947
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-17
#  [*] Finished testing Best Epoch: 17 , accuracy:  0.9541666507720947 !

Training: 8000
CNN:x (80, 128, 128, 3)
CNN: (80, 128, 128, 32)
CNN: (80, 64, 64, 32)
CNN: (80, 64, 64, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 8, 8, 64)
CNN: (80, 8, 8, 128)
CNN: (80, 4, 4, 128)
CNN: (80, 2048)
CNN:out_logit (80, 4)
------------------------
CNN:x (80, 128, 128, 3)
CNN: (80, 128, 128, 32)
CNN: (80, 64, 64, 32)
CNN: (80, 64, 64, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 32, 32, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 16, 16, 64)
CNN: (80, 8, 8, 64)
CNN: (80, 8, 8, 128)
CNN: (80, 4, 4, 128)
CNN: (80, 2048)
CNN:out_logit (80, 4)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [18432, bytes: 73728]
cnn/conv

Epoch: [ 2] [   6/ 100] time: 14.9533, loss: 0.51194912
Epoch: [ 2] [   7/ 100] time: 15.0689, loss: 0.48113635
Epoch: [ 2] [   8/ 100] time: 15.1854, loss: 0.52173805
Epoch: [ 2] [   9/ 100] time: 15.3010, loss: 0.42552489
Epoch: [ 2] [  10/ 100] time: 15.4175, loss: 0.44586921
Epoch: [ 2] [  11/ 100] time: 15.5333, loss: 0.29458088
Epoch: [ 2] [  12/ 100] time: 15.6491, loss: 0.44293791
Epoch: [ 2] [  13/ 100] time: 15.7646, loss: 0.38150442
Epoch: [ 2] [  14/ 100] time: 15.8803, loss: 0.31562036
Epoch: [ 2] [  15/ 100] time: 15.9960, loss: 0.50863606
Epoch: [ 2] [  16/ 100] time: 16.1116, loss: 0.43607020
Epoch: [ 2] [  17/ 100] time: 16.2281, loss: 0.35626113
Epoch: [ 2] [  18/ 100] time: 16.3437, loss: 0.28144994
Epoch: [ 2] [  19/ 100] time: 16.4603, loss: 0.34983081
Epoch: [ 2] [  20/ 100] time: 16.5759, loss: 0.34399068
Epoch: [ 2] [  21/ 100] time: 16.6913, loss: 0.49747586
Epoch: [ 2] [  22/ 100] time: 16.8067, loss: 0.38260216
Epoch: [ 2] [  23/ 100] time: 16.9224, loss: 0.3

Epoch: [ 3] [  52/ 100] time: 32.9486, loss: 0.24217895
Epoch: [ 3] [  53/ 100] time: 33.0645, loss: 0.36824635
Epoch: [ 3] [  54/ 100] time: 33.1820, loss: 0.33276302
Epoch: [ 3] [  55/ 100] time: 33.2978, loss: 0.38985276
Epoch: [ 3] [  56/ 100] time: 33.4150, loss: 0.16712523
Epoch: [ 3] [  57/ 100] time: 33.5315, loss: 0.19295374
Epoch: [ 3] [  58/ 100] time: 33.6474, loss: 0.25461155
Epoch: [ 3] [  59/ 100] time: 33.7632, loss: 0.38311106
Epoch: [ 3] [  60/ 100] time: 33.8794, loss: 0.26622313
Epoch: [ 3] [  61/ 100] time: 33.9960, loss: 0.20752123
Epoch: [ 3] [  62/ 100] time: 34.1129, loss: 0.18123969
Epoch: [ 3] [  63/ 100] time: 34.2295, loss: 0.38729560
Epoch: [ 3] [  64/ 100] time: 34.3457, loss: 0.16751274
Epoch: [ 3] [  65/ 100] time: 34.4629, loss: 0.25078678
Epoch: [ 3] [  66/ 100] time: 34.5789, loss: 0.35803428
Epoch: [ 3] [  67/ 100] time: 34.6955, loss: 0.31511587
Epoch: [ 3] [  68/ 100] time: 34.8111, loss: 0.24415630
Epoch: [ 3] [  69/ 100] time: 34.9270, loss: 0.3

Epoch: [ 4] [  98/ 100] time: 51.2103, loss: 0.08322574
Epoch: [ 4] [  99/ 100] time: 51.3365, loss: 0.29746968
[4/50] - ptime: 12.5752 loss: 0.21314768 acc: 0.85833 lr: 0.00095873
Epoch: [ 5] [   0/ 100] time: 52.4924, loss: 0.10585015
Epoch: [ 5] [   1/ 100] time: 52.6072, loss: 0.14109929
Epoch: [ 5] [   2/ 100] time: 52.7241, loss: 0.11846907
Epoch: [ 5] [   3/ 100] time: 52.8405, loss: 0.16893771
Epoch: [ 5] [   4/ 100] time: 52.9560, loss: 0.25662589
Epoch: [ 5] [   5/ 100] time: 53.0736, loss: 0.23165971
Epoch: [ 5] [   6/ 100] time: 53.1900, loss: 0.06445827
Epoch: [ 5] [   7/ 100] time: 53.3071, loss: 0.17497481
Epoch: [ 5] [   8/ 100] time: 53.4237, loss: 0.17195028
Epoch: [ 5] [   9/ 100] time: 53.5403, loss: 0.08312681
Epoch: [ 5] [  10/ 100] time: 53.6588, loss: 0.13230200
Epoch: [ 5] [  11/ 100] time: 53.7799, loss: 0.10969331
Epoch: [ 5] [  12/ 100] time: 53.9039, loss: 0.20675923
Epoch: [ 5] [  13/ 100] time: 54.0286, loss: 0.19475472
Epoch: [ 5] [  14/ 100] time: 54.15

Epoch: [ 6] [  42/ 100] time: 70.3803, loss: 0.09742427
Epoch: [ 6] [  43/ 100] time: 70.4965, loss: 0.17221430
Epoch: [ 6] [  44/ 100] time: 70.6127, loss: 0.15791783
Epoch: [ 6] [  45/ 100] time: 70.7291, loss: 0.09147124
Epoch: [ 6] [  46/ 100] time: 70.8453, loss: 0.11143366
Epoch: [ 6] [  47/ 100] time: 70.9614, loss: 0.09943257
Epoch: [ 6] [  48/ 100] time: 71.0777, loss: 0.07357304
Epoch: [ 6] [  49/ 100] time: 71.1935, loss: 0.06216942
Epoch: [ 6] [  50/ 100] time: 71.3117, loss: 0.07997140
Epoch: [ 6] [  51/ 100] time: 71.4290, loss: 0.04248011
Epoch: [ 6] [  52/ 100] time: 71.5453, loss: 0.09462978
Epoch: [ 6] [  53/ 100] time: 71.6611, loss: 0.10961495
Epoch: [ 6] [  54/ 100] time: 71.7772, loss: 0.08123219
Epoch: [ 6] [  55/ 100] time: 71.8934, loss: 0.17929646
Epoch: [ 6] [  56/ 100] time: 72.0098, loss: 0.16116346
Epoch: [ 6] [  57/ 100] time: 72.1261, loss: 0.19093284
Epoch: [ 6] [  58/ 100] time: 72.2434, loss: 0.13677661
Epoch: [ 6] [  59/ 100] time: 72.3594, loss: 0.1

...

Epoch: [45] [  10/ 100] time: 572.3274, loss: 0.01940287
Epoch: [45] [  11/ 100] time: 572.4451, loss: 0.00353343
Epoch: [45] [  12/ 100] time: 572.5616, loss: 0.00142746
Epoch: [45] [  13/ 100] time: 572.6792, loss: 0.01044926
Epoch: [45] [  14/ 100] time: 572.7959, loss: 0.00219584
Epoch: [45] [  15/ 100] time: 572.9132, loss: 0.06203147
Epoch: [45] [  16/ 100] time: 573.0298, loss: 0.00307803
Epoch: [45] [  17/ 100] time: 573.1482, loss: 0.00411007
Epoch: [45] [  18/ 100] time: 573.2648, loss: 0.00089341
Epoch: [45] [  19/ 100] time: 573.3829, loss: 0.00598542
Epoch: [45] [  20/ 100] time: 573.4996, loss: 0.00173625
Epoch: [45] [  21/ 100] time: 573.6164, loss: 0.02618721
Epoch: [45] [  22/ 100] time: 573.7328, loss: 0.00236021
Epoch: [45] [  23/ 100] time: 573.8505, loss: 0.00559454
Epoch: [45] [  24/ 100] time: 573.9673, loss: 0.02332485
Epoch: [45] [  25/ 100] time: 574.0836, loss: 0.02139883
Epoch: [45] [  26/ 100] time: 574.2018, loss: 0.00028186
Epoch: [45] [  27/ 100] time: 5

Epoch: [46] [  54/ 100] time: 590.2242, loss: 0.01500958
Epoch: [46] [  55/ 100] time: 590.3417, loss: 0.00355528
Epoch: [46] [  56/ 100] time: 590.4598, loss: 0.00014741
Epoch: [46] [  57/ 100] time: 590.5774, loss: 0.01505694
Epoch: [46] [  58/ 100] time: 590.6949, loss: 0.00262816
Epoch: [46] [  59/ 100] time: 590.8112, loss: 0.00082432
Epoch: [46] [  60/ 100] time: 590.9279, loss: 0.00152075
Epoch: [46] [  61/ 100] time: 591.0449, loss: 0.00030306
Epoch: [46] [  62/ 100] time: 591.1624, loss: 0.00099294
Epoch: [46] [  63/ 100] time: 591.2796, loss: 0.00037574
Epoch: [46] [  64/ 100] time: 591.3969, loss: 0.00066254
Epoch: [46] [  65/ 100] time: 591.5140, loss: 0.00489669
Epoch: [46] [  66/ 100] time: 591.6311, loss: 0.00096354
Epoch: [46] [  67/ 100] time: 591.7483, loss: 0.00146292
Epoch: [46] [  68/ 100] time: 591.8653, loss: 0.00010141
Epoch: [46] [  69/ 100] time: 591.9831, loss: 0.00328869
Epoch: [46] [  70/ 100] time: 592.1002, loss: 0.00085480
Epoch: [46] [  71/ 100] time: 5

Epoch: [47] [  98/ 100] time: 608.1815, loss: 0.00006129
Epoch: [47] [  99/ 100] time: 608.2984, loss: 0.00152028
[47/50] - ptime: 12.4048 loss: 0.00355279 acc: 0.94583 lr: 0.00060945
Epoch: [48] [   0/ 100] time: 609.4462, loss: 0.00299767
Epoch: [48] [   1/ 100] time: 609.5611, loss: 0.00031071
Epoch: [48] [   2/ 100] time: 609.6776, loss: 0.00014838
Epoch: [48] [   3/ 100] time: 609.7945, loss: 0.00235947
Epoch: [48] [   4/ 100] time: 609.9113, loss: 0.00721442
Epoch: [48] [   5/ 100] time: 610.0281, loss: 0.00036556
Epoch: [48] [   6/ 100] time: 610.1460, loss: 0.00004935
Epoch: [48] [   7/ 100] time: 610.2641, loss: 0.00026222
Epoch: [48] [   8/ 100] time: 610.3813, loss: 0.00145317
Epoch: [48] [   9/ 100] time: 610.4983, loss: 0.00093165
Epoch: [48] [  10/ 100] time: 610.6149, loss: 0.00065836
Epoch: [48] [  11/ 100] time: 610.7316, loss: 0.00003584
Epoch: [48] [  12/ 100] time: 610.8481, loss: 0.00007847
Epoch: [48] [  13/ 100] time: 610.9648, loss: 0.00016572
Epoch: [48] [  14/

Epoch: [49] [  40/ 100] time: 626.8827, loss: 0.00060403
Epoch: [49] [  41/ 100] time: 626.9990, loss: 0.00023408
Epoch: [49] [  42/ 100] time: 627.1159, loss: 0.00010576
Epoch: [49] [  43/ 100] time: 627.2342, loss: 0.00022511
Epoch: [49] [  44/ 100] time: 627.3523, loss: 0.00451050
Epoch: [49] [  45/ 100] time: 627.4708, loss: 0.00133127
Epoch: [49] [  46/ 100] time: 627.5877, loss: 0.00006882
Epoch: [49] [  47/ 100] time: 627.7054, loss: 0.00013272
Epoch: [49] [  48/ 100] time: 627.8225, loss: 0.00009516
Epoch: [49] [  49/ 100] time: 627.9389, loss: 0.00025911
Epoch: [49] [  50/ 100] time: 628.0565, loss: 0.00063373
Epoch: [49] [  51/ 100] time: 628.1744, loss: 0.00127328
Epoch: [49] [  52/ 100] time: 628.2935, loss: 0.00014143
Epoch: [49] [  53/ 100] time: 628.4119, loss: 0.00005771
Epoch: [49] [  54/ 100] time: 628.5291, loss: 0.00043919
Epoch: [49] [  55/ 100] time: 628.6459, loss: 0.00010150
Epoch: [49] [  56/ 100] time: 628.7626, loss: 0.00006315
Epoch: [49] [  57/ 100] time: 6

Epoch: [50] [  84/ 100] time: 644.8243, loss: 0.00067153
Epoch: [50] [  85/ 100] time: 644.9409, loss: 0.00007160
Epoch: [50] [  86/ 100] time: 645.0574, loss: 0.00080791
Epoch: [50] [  87/ 100] time: 645.1741, loss: 0.00794054
Epoch: [50] [  88/ 100] time: 645.2937, loss: 0.00037992
Epoch: [50] [  89/ 100] time: 645.4128, loss: 0.00020158
Epoch: [50] [  90/ 100] time: 645.5295, loss: 0.01265701
Epoch: [50] [  91/ 100] time: 645.6466, loss: 0.00430959
Epoch: [50] [  92/ 100] time: 645.7631, loss: 0.00008458
Epoch: [50] [  93/ 100] time: 645.8826, loss: 0.00020037
Epoch: [50] [  94/ 100] time: 645.9976, loss: 0.00014918
Epoch: [50] [  95/ 100] time: 646.1124, loss: 0.00002476
Epoch: [50] [  96/ 100] time: 646.2306, loss: 0.00003000
Epoch: [50] [  97/ 100] time: 646.3571, loss: 0.00964777
Epoch: [50] [  98/ 100] time: 646.4796, loss: 0.00244160
Epoch: [50] [  99/ 100] time: 646.5978, loss: 0.00002069
[50/50] - ptime: 12.4046 loss: 0.00198569 acc: 0.93750 lr: 0.00059049
Avg per epoch ptim

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放